## Demonstracja użycia biblioteki cloudberry.plots do tworzenia wykresów.

## 1.1 Podstawowe użycie
Celem tworzenia wykresu należy stworzyć obiekt z konfiguracją wykresu (nazwy osi, legenda, nazwa wykresu i inne) - PlotProperties.
Następnie właściwy wykres tworzymy przy użyciu API klasy PlotBuilder.

In [ ]:
import cloudberry.plots as cp
import cloudberry.api.model as model

properties = cp.PlotProperties.default()
properties.default_series_kind = cp.PlotSeriesKind.LINE
properties.show_legend = True
properties.x_axis_name = 'Time [s]'
properties.y_axis_name = 'Values'
builder = cp.PlotBuilder(properties)

Przykładowe dane do wyświetlenia.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'time': [1, 2, 3, 4],
    'value': [2, 1, 3, 7],
    'value_error': [.2, .1, .3, .7],
})
df

Z powyższych danych tworzymy PlotSeries - obiekt reprezentujący pojedynczą serię wykresu i dodajemy go do PlotBuilder'a.

In [ ]:
serie_id = "1st seriesId"
serie1 = cp.PlotSeries(
    series_info=model.SeriesInfo("1st series", serie_id), 
    data=df,
    x_field="time", 
    y_field="value",
    y_err_field="value_error",
)

builder.add_series(serie1)

Poniżej tworzymy i wyświetlamy wykres.

In [ ]:
plot = builder.plot()

In [ ]:
plot.show()

## 1.2. Dodanie linii trendu
Biblioteka oferuje kilka typów linii trendu:
* regresja metodą KNN (K Nearest Neighbors)
* regresja liniowa
* funkcja stała (do zaznaczenia wartości progowych)

Regresja KNN:

In [ ]:
builder.add_trend('KNN Regression', cp.PlotlyTrendLine(serie_id, cp.PlotlyTrendLineKind.KNN10))
plot_with_trend = builder.plot()
plot_with_trend

Regresja liniowa:

In [ ]:
builder.add_trend('Linear Regression', cp.PlotlyTrendLine(serie_id, cp.PlotlyTrendLineKind.LINEAR))
plot_with_trend = builder.plot()
plot_with_trend

Funkcja stała:

In [ ]:
builder.add_trend('Const Threshold', cp.PlotlyTrendLine(serie_id, cp.PlotlyTrendLineKind.CONST, constant=6))
plot_with_trend = builder.plot()
plot_with_trend

## 1.3. Użycie biblioteki na danych pobranych z Cloudberry

In [ ]:
import cloudberry.api as cb

cb_config = cb.CloudberryConfig(f'http://localhost:9000')

In [ ]:
cb_meta = cb.Metadata(cb_config)
cb_meta_experiment_computation = cb_meta.experiment_computation_api()

computation = cb_meta_experiment_computation.find_all()[0]

In [ ]:
x_field = '_time'
y_field = 'AVERAGE_FITNESS'
cb_computation_series_api = cb.ComputationSeries(cb_config)

result = cb_computation_series_api.for_computations([computation], y_field)
data_series = result.series[0]

plot_series = cp.PlotSeries(name = data_series.series_name, 
                            data = data_series.as_data_frame,
                            x_field = x_field,
                            y_field = y_field,
                            y_err_field = None)

In [ ]:
plot_builder = cp.PlotBuilder(properties)
plot_builder.add_series(plot_series)
plot_builder.plot()